# Get Lat Lon Port Indo
Mendapatkan data port Indonesia beserta koordinat pusatnya (latitude & longitude) dari web https://msi.nga.mil/

Pertama, mengimpor beberapa fungsi dari library di chunk di bawah

In [15]:
#lib
import time          #time
import re            #replace string
import pandas as pd  #data manipulation
from dms2dec.dms_convert import dms2dec #convert coordinate type
from urllib.parse import urljoin        #convert relative url to abs url
from csv import writer                  #write to csv
#scraping library
from fake_useragent import UserAgent    #fake useragent
from bs4 import BeautifulSoup           #pull data from html
#library selenium for scraping 2
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

## Beberapa Function
Fungsi buka_browser berfungsi untuk membuka browser secara otomatis dan go to web dari link yang diberikan. Browser yang dipakai adalah Firefox. UserAgent digunakan untuk memalsukan user agent agar tidak diblok saat melakukan scraping.

In [16]:
def buka_browser(link):
    '''Membuka browser secara otomatis dan go to web dari link yang diberikan'''
    # FAKE USER AGENT
    ua = UserAgent()
    user_agent = ua.chrome
    print('User_agent:', user_agent)
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", user_agent)

    # HEADLESS FIREFOX
    opts = Options()
    #opts.headless = True
    #assert opts.headless

    # GO TO WEB
    print('Status: Membuka Firefox')
    driver = webdriver.Firefox(executable_path='C:/Users/asus/Documents/Skripsi/geckodriver.exe', firefox_profile=profile, options = opts)
    print('Status: Membuka website')
    driver.get(link)
    print('Status: Sudah buka, menunggu')
    
    return(driver)

In [17]:
def tambah_data_row(nama_file, list_isi):
    '''Menambah data ke file csv'''
    # buka file dalam mode append
    with open(nama_file, 'a+', newline='') as file:
        # membuat modul writer dari lib
        csv_writer = writer(file)
        # add isi pada baris terakhir data
        csv_writer.writerow(list_isi)

## Exekusi
Mengimplementasikan fungsi di atas untuk mengambil data dari web. Pertama, membuka browser dan menuju link https://msi.nga.mil/ yang memuat daftar port Indonesia.

In [18]:
# open driver n open web
link = 'https://msi.nga.mil/queryResults?publications/world-port-index?countryName=Indonesia&output=html'
driver = buka_browser(link)
time.sleep(10)

User_agent: Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36
Status: Membuka Firefox
Status: Membuka website
Status: Sudah buka, menunggu


Proses kedua, yaitu mengambil data di page html yang terbuka. Page html yang terbuka memuat daftar port Indonesia yang berbentuk link. Apabila dikunjungi, maka akan menampilkan detail informasi tentang port tersebut.

In [19]:
start_url = "https://msi.nga.mil/"
# konversi ke soup object
soup_page = BeautifulSoup(driver.page_source, 'lxml')
# dapetin isi tabel
tbl = soup_page.find('table', class_='port')
# dapetin link masing-masing port dalam tabel
a_tags = tbl.find_all('a')
links = [urljoin(start_url, a['href'])for a in a_tags]  # convert relative url to absolute url
#print(links)

Setelah link port didapatkan, maka satu per satu driver akan mengunjungi link tersebut untuk mengekstrak informasi detail suatu port. Informasi yang diambil yaitu:
- Nama port
- Nomor port
- Koordinat yang meliputi latitude dan longitude

In [ ]:
all_data = []
i=1
# click semua link
for link in links:
    # menuju page per link
    driver.get(link)
    time.sleep(7)
    
    # load page
    link_page = BeautifulSoup(driver.page_source, 'lxml')
    # PORT
    port = link_page.find_all('td', class_='case-capitalize')[1]
    ## port name
    port_name = port.find('b').text
    ## port no
    a,b,c = port.text.partition('No. ')
    port_no = re.sub(r'\([^)]*\)','',c)
    # COORDINATE
    td = link_page.find_all('td', class_='port')
    ## lat
    port_lat = re.sub('Latitude: ','',td[1].text)
    ## lon
    port_lon = re.sub('Longitude: ','',td[3].text)
    
    # masukin ke file csv
    #print(port_name)
    tambah_data_row('port.csv',[port_no, port_name, port_lat, port_lon])

In [21]:
# Quit driver ketika selesai mengambil data
driver.quit()

# Preproses Data
Data yang telah diambil akan dilakukan beberapa penyesuaian, yaitu konversi koordinat DDM (Decimal Minute) ke koordinate DD (Decimal Degree)

In [9]:
# Read data
df = pd.read_csv('port-raw.csv',encoding="ISO-8859-1", names=['no','name','lat_ddm','lon_ddm'])
print('Jumlah data: ',len(df))
df.head()

Jumlah data:  123


,no,name,lat_ddm,lon_ddm
0,53115,AMAMAPARE,4° 49' S,136° 58' E
1,52720,AMBON,3° 41' S,128° 10' E
2,51280,AMPENAN,8° 34' S,116° 4' E
3,50930,ANYER LOR,6° 3' S,105° 55' E
4,50340,ARDJUNA OIL FIELD,5° 53' S,107° 43' E


## Convert latlon (DDM to DD)

Use this package <br>
`dms2dec('''36°44'47.69"N''') #converts to dec`

Contoh hasil convert <br/>
`Jakarta, lat: 6° 6' S, lon: 106° 53' E` <br/>
Berubah jadi <br/>
`Jakarta, lat: -6.1, lon: 106.88333`

In [97]:
# Konversi latitude
df['lat_dd'] = df.lat_ddm.apply(lambda x: dms2dec(x))
# Konversi longitude
df['lon_dd'] = df.lon_ddm.apply(lambda x: dms2dec(x))
df.head()

,no,name,lat_ddm,lon_ddm,lat_dd,lon_dd
0,53115,AMAMAPARE,4° 49' S,136° 58' E,-4.816667,136.966667
1,52720,AMBON,3° 41' S,128° 10' E,-3.683333,128.166667
2,51280,AMPENAN,8° 34' S,116° 4' E,-8.566667,116.066667
3,50930,ANYER LOR,6° 3' S,105° 55' E,-6.050000,105.916667
4,50340,ARDJUNA OIL FIELD,5° 53' S,107° 43' E,-5.883333,107.716667


In [10]:
# Menyimpan data ke csv
df.to_csv('port.csv')

# Coret

In [68]:
link_page = BeautifulSoup(driver.page_source, 'lxml')
a = link_page.find_all('td', class_='port')
a[1] #lat

<td class="port" width="50%"><b>Latitude: </b>4° 49' S</td>

In [73]:
a[3].text #lon
re.sub('Longitude: |Latitude: ','',a[3].text)

"4° 49' S"

In [92]:
port = link_page.find_all('td', class_='case-capitalize')[1]
port_name = port.find('b').text
print(port_name)
a,b,c = port.text.partition('No. ')
port_no = re.sub(r'\([^)]*\)','',c)
print(port_no)

AMAMAPARE
53115


In [ ]:
start_url = "https://msi.nga.mil/"
get_links(start_url)

a = "queryBreakout?publications/world-port-index?indexNumber=53115&amp;output=html/"
print('Status: Masuk web')
a = re.sub(r"amp;",'',a)
urljoin(start_url, a)
